## **1.9_DATASET_masters_compare_quality_columns.ipynb**

### GOALS of this script:
* use the filtered_XXX.csv
* compare qualitx columns for all datasets based on the ID column
    * making a barplot of the number of markers that have the same quality class across all datasets
    * making a barplot for the non-optimal marker classes within each dataset
    * makeing a barplot for the comparison for the best quality categories vs. all other categorie
* OUT: three graphs as png 

### *Import packages*

In [ ]:
import gzip
import allel
import pandas as pd
import numpy as np
import tskit
import tsinfer
import sys
import json
import csv
from IPython.display import SVG
from IPython.display import HTML
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from openpyxl import Workbook
from progressbar import ProgressBar
sns.set_style('white')
sns.set_style('ticks')
import matplotlib.ticker as mticker

## *Import files*

**Load filtered datasets**

In [ ]:
filtered_unterseer_2016_elite=pd.read_csv("filtered_unterseer_2016_elite_for_masters.csv", sep="\t")

In [ ]:
filtered_unterseer_2016_elite

In [ ]:
filtered_unterseer_2016_landraces=pd.read_csv("filtered_unterseer_2016_landraces_for_masters.csv", sep="\t")

In [ ]:
filtered_unterseer_2016_landraces

In [ ]:
filtered_mayer_2017_landraces=pd.read_csv("filtered_mayer_2017_landraces_for_masters.csv", sep="\t")

In [ ]:
filtered_mayer_2017_landraces

In [ ]:
filtered_mayer_2020_dhlines=pd.read_csv("filtered_mayer_2020_dhlines_for_masters.csv", sep="\t")

In [ ]:
filtered_mayer_2020_dhlines

## **1.9 Compare quality columns between all datasets & make bar plots**

### **1.9.1 Get the marker and the quality column from each dataset**

**Unterseer_2016 - Elite**

In [ ]:
#take the table where the non-typical genotype calls are removed already
quality_select_unterseer_2016_elite=filtered_unterseer_2016_elite[["variants/AD_2016_elite","variants/CMT_2016_elite"]]
quality_select_unterseer_2016_elite=quality_select_unterseer_2016_elite.rename(columns={"variants/AD_2016_elite": "marker_ID"})
quality_select_unterseer_2016_elite

**Unterseer_2016 - Landraces**

In [ ]:
#take the table where the non-typical genotype calls are removed already
quality_select_unterseer_2016_landraces=filtered_unterseer_2016_landraces[["variants/AD_2016_landraces","variants/CMT_2016_landraces"]]
quality_select_unterseer_2016_landraces=quality_select_unterseer_2016_landraces.rename(columns={"variants/AD_2016_landraces": "marker_ID"})
quality_select_unterseer_2016_landraces

**Mayer_2017_landraces**

In [ ]:
##take the table where the non-typical genotype calls are removed already
quality_select_mayer_2017_landraces=filtered_mayer_2017_landraces[["index_2017_landraces","quality_2017_landraces"]]
quality_select_mayer_2017_landraces=quality_select_mayer_2017_landraces.rename(columns={"index_2017_landraces": "marker_ID"})
quality_select_mayer_2017_landraces

**Mayer_2020_dhlines**

In [ ]:
##take the table where the non-typical genotype calls are removed already
quality_select_mayer_2020_dhlines=filtered_mayer_2020_dhlines[["marker_2020_dh","quality_2020_dh"]]
quality_select_mayer_2020_dhlines=quality_select_mayer_2020_dhlines.rename(columns={"marker_2020_dh":"marker_ID"})
quality_select_mayer_2020_dhlines

### **1.9.2 Merge all the marker quality columns into one table**

In [ ]:
#Merge the Unterseer columns
merge_marker_unterseer=pd.merge(quality_select_unterseer_2016_elite,quality_select_unterseer_2016_landraces, on='marker_ID', how='outer')
merge_marker_unterseer

In [ ]:
#Merge the Mayer columns 
merge_marker_mayer=pd.merge(quality_select_mayer_2020_dhlines,quality_select_mayer_2017_landraces, on='marker_ID', how='outer')
merge_marker_mayer

In [ ]:
#Merge the two merge tables  
merge_marker_all=pd.merge(merge_marker_mayer,merge_marker_unterseer, on='marker_ID', how='outer')
merge_marker_all

*Check which markers are in the columns*

In [ ]:
check_marker_all=merge_marker_all[["quality_2020_dh","quality_2017_landraces","variants/CMT_2016_elite","variants/CMT_2016_landraces"]].astype(str)

In [ ]:
counter_marker_all=check_marker_all.values.ravel()

In [ ]:
Counter(counter_marker_all)

In [ ]:
609442-609132 #difference in length between the Unterseer and Mayer is 310 lines, two row are missing 310 lines therefore 620 "nan" make sense 

In [ ]:
unique_marker_all=pd.unique(check_marker_all.values.ravel())

In [ ]:
Counter(unique_marker_all)

In [ ]:
unique_marker_all=pd.unique(check_marker_all.values.ravel())

In [ ]:
Counter(unique_marker_all)

### **1.9.2 Count the occurences of each marker per dataset**

*Count occurences in dataset for the bar plot* - **Unterseer_2016_elite**

In [ ]:
unterseer_2016_elite_quality_count=quality_select_unterseer_2016_elite['variants/CMT_2016_elite'].value_counts()

In [ ]:
unterseer_2016_elite_quality_count

In [ ]:
#609132
560483+45387+2572+387+273+30

In [ ]:
unterseer_2016_elite_quality_count_list=unterseer_2016_elite_quality_count.tolist()

In [ ]:
#tolist for plotting in the bargraph 
unterseer_2016_elite_quality_count_list

*Count occurences in dataset for the bar plot* - **Unterseer_2016_landraces**

In [ ]:
unterseer_2016_landraces_quality_count=quality_select_unterseer_2016_landraces['variants/CMT_2016_landraces'].value_counts()

In [ ]:
unterseer_2016_landraces_quality_count

In [ ]:
#609132
527511+60954+11392+7908+846+521

*Count occurences in dataset for the bar plot* - **Mayer_2016_landraces**

In [ ]:
mayer_2017_landraces_quality_count=quality_select_mayer_2017_landraces["quality_2017_landraces"].value_counts()

In [ ]:
mayer_2017_landraces_quality_count

In [ ]:
#609442
527743+61023+11395+7912+848+521

*Count occurences in dataset for the bar plot* - **Mayer_2020_dhlines**

In [ ]:
mayer_2020_dhlines_quality_count=quality_select_mayer_2020_dhlines["quality_2020_dh"].value_counts()

In [ ]:
mayer_2020_dhlines_quality_count

In [ ]:
#609442
523955+47683+20258+15740+1058+748

In [ ]:
Counter(unique_marker_all)

### **1.9.3 Count the quality marker combinations for each marker across all datasets**

In [ ]:
polyhigh_check=merge_marker_all[(merge_marker_all["quality_2020_dh"]=="PolyHighResolution")&(merge_marker_all["quality_2017_landraces"]=="PolyHighResolution")&(merge_marker_all["variants/CMT_2016_elite"]=="PolyHighResolution")&(merge_marker_all["variants/CMT_2016_landraces"]=="PolyHighResolution")]

In [ ]:
len(polyhigh_check)

In [ ]:
OTV_check=merge_marker_all[(merge_marker_all["quality_2020_dh"]=="OTV")&(merge_marker_all["quality_2017_landraces"]=="OTV")&(merge_marker_all["variants/CMT_2016_elite"]=="OTV")&(merge_marker_all["variants/CMT_2016_landraces"]=="OTV")]

In [ ]:
len(OTV_check["marker_ID"])

In [ ]:
Other_check=merge_marker_all[(merge_marker_all["quality_2020_dh"]=="Other")&(merge_marker_all["quality_2017_landraces"]=="Other")&(merge_marker_all["variants/CMT_2016_elite"]=="Other")&(merge_marker_all["variants/CMT_2016_landraces"]=="Other")]

In [ ]:
len(Other_check)

In [ ]:
MonoHighResolution_check=merge_marker_all[(merge_marker_all["quality_2020_dh"]=="MonoHighResolution")&(merge_marker_all["quality_2017_landraces"]=="MonoHighResolution")&(merge_marker_all["variants/CMT_2016_elite"]=="MonoHighResolution")&(merge_marker_all["variants/CMT_2016_landraces"]=="MonoHighResolution")]

In [ ]:
len(MonoHighResolution_check)

In [ ]:
NoMinorHom_check=merge_marker_all[(merge_marker_all["quality_2020_dh"]=="NoMinorHom")&(merge_marker_all["quality_2017_landraces"]=="NoMinorHom")&(merge_marker_all["variants/CMT_2016_elite"]=="NoMinorHom")&(merge_marker_all["variants/CMT_2016_landraces"]=="NoMinorHom")]

In [ ]:
len(NoMinorHom_check)

In [ ]:
CallRateBelowThreshold_check=merge_marker_all[(merge_marker_all["quality_2020_dh"]=="CallRateBelowThreshold")&(merge_marker_all["quality_2017_landraces"]=="CallRateBelowThreshold")&(merge_marker_all["variants/CMT_2016_elite"]=="CallRateBelowThreshold")&(merge_marker_all["variants/CMT_2016_landraces"]=="CallRateBelowThreshold")]

In [ ]:
len(CallRateBelowThreshold_check)

In [ ]:
nan_check=merge_marker_all[(merge_marker_all["quality_2020_dh"]=="nan")&(merge_marker_all["quality_2017_landraces"]=="nan")&(merge_marker_all["variants/CMT_2016_elite"]=="nan")&(merge_marker_all["variants/CMT_2016_landraces"]=="nan")]

In [ ]:
len(nan_check)

### **1.9.4 Count distribution of row-wise number of markers & number of all the same marker across**

In [ ]:
#take the marker qualities across all datasets
merge_marker_all=merge_marker_all.drop(columns=['marker_ID'])

In [ ]:
#make a list of sets out of them 
merge_marker_all_set=list(map(set,merge_marker_all.values))

In [ ]:
#merge_marker_all_set

In [ ]:
#make a list of lists of out them 
merge_marker_all_list=[]

In [ ]:
for s in merge_marker_all_set:
    #print(e)
    quality_row=list(s)
    merge_marker_all_list.append(quality_row)

In [ ]:
#merge_marker_all_list

In [ ]:
#number of quality marker list
one_marker=0
two_markers=0
three_markers=0
four_markers=0

In [ ]:
#counter for all the same marker across 
counter_PolyHighResolution_across=0
counter_NoMinorHom_across=0
counter_MonoHighResolution_across=0
counter_OTV_across=0
counter_CallRateBelowThreshold_across=0
counter_Other_across=0
counter_nan_across=0

In [ ]:
for merge_marker in merge_marker_all_list:
    #print(merge_marker)
    if len(merge_marker)==1:# if there is only one quality == all the same 
        one_marker=one_marker+1
        #print("One quality marker: " + str(merge_marker))
        for element_marker in merge_marker:
            if element_marker == "PolyHighResolution": # best quality across all datasets - keep
                #print(element_marker + ": best quality across all")
                counter_PolyHighResolution_across = counter_PolyHighResolution_across+1
            if element_marker == "NoMinorHom": #second best quality acorss all datasets - keep 
                #print(element_marker + ": second best quality across all")
                counter_NoMinorHom_across = counter_NoMinorHom_across+1
            if element_marker == "MonoHighResolution": #might be homozygous across all datasets - keep for now 
                #print(element_marker + ": might be homozygous across all - keep for now")
                counter_MonoHighResolution_across = counter_MonoHighResolution_across+1
            if element_marker == "OTV": 
                #print(element_marker + ": other target - remove")
                counter_OTV_across = counter_OTV_across+1
            if element_marker == "CallRateBelowThreshold": # if the marker quality across all the datasets is the second lowest quality, remove row (append FALSE)
                #print(element_marker + ": second lowest quality across all")
                counter_CallRateBelowThreshold_across = counter_CallRateBelowThreshold_across+1
            if element_marker == "Other": #if the marker quality across all datasets is the lowest quality, remove row (append FALSE)
                #print(element_marker + ": lowest quality across all")
                counter_Other_across = counter_Other_across+1
            if element_marker == "nan": #if the marker quality across all datasets is missing something is wrong - STOP
                counter_nan_across = counter_nan_across+1
                break
    if len(merge_marker)==2:
        two_markers=two_markers+1
        #print("Two quality markers: " + str(merge_marker))
    if len(merge_marker)==3:
        three_markers=three_markers+1
        #print("Three quality markers: " + str(merge_marker))
    if len(merge_marker)==4:
        four_markers=four_markers+1
        #print("Four quality markers: " + str(merge_marker))

In [ ]:
#same marker across all datasets 
print("counter_PolyHighResolution_across: " + str(counter_PolyHighResolution_across))
print("counter_NoMinorHom_across: " + str(counter_NoMinorHom_across))
print("counter_MonoHighResolution_across: " + str(counter_MonoHighResolution_across))
print("counter_OTV_across: " + str(counter_OTV_across))
print("counter_CallRateBelowThreshold_across: " + str(counter_CallRateBelowThreshold_across))
print("counter_Other_across: " + str(counter_Other_across))
print("counter_nan_across: " + str(counter_nan_across))
across_sum=counter_PolyHighResolution_across+counter_NoMinorHom_across+counter_MonoHighResolution_across+counter_OTV_across+counter_CallRateBelowThreshold_across+counter_Other_across+counter_nan_across
print("SUM:" + str(across_sum))

In [ ]:
#row-wise number of markers
print("one_marker: " + str(one_marker))
print("two_markers: " + str(two_markers))
print("three_markers: " + str(three_markers))
print("four_markers: " + str(four_markers))
row_wise_sum=one_marker+two_markers+three_markers+four_markers
print("SUM:" + str(row_wise_sum))

### **1.9.5 Make barplots**

In [ ]:
#permanantly change the display fig size in the jupyter notebook
plt.rcParams["figure.figsize"]=50,25
plt.rcParams.update({'font.size': 35})

#### **1.9.5.1 Making a barplot of the number of markers that have the same quality class across all datasets**

In [ ]:
labels_aa = ['OTV', 'Other', 'MonoHighResolution', 'NoMinorHom', 'CallRateBelowThreshold', 'nan']
across_number=[7112, 735, 146, 53, 0, 0]


fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(labels_aa,across_number)

#Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('number of markers',fontsize='large', fontweight= "bold")
ax.set_title('Axiom Array 600k Marker Quality Category - same across all datasets', fontsize='xx-large', fontweight= "bold")

plt.show()
#plt.savefig('/Users/kschul38/Documents/tsinfer-project/figures/Stetter_phd_europe_maize_figures/same_across_all_datasets.png')

#### **1.9.5.2 Making a barplot for the non-optimal marker classes**

*Convert the datasetwise counts into dataframes*

In [ ]:
unterseer_2016_landraces_quality_count_table=unterseer_2016_landraces_quality_count.to_frame()
#unterseer_2016_landraces_quality_count_table

In [ ]:
unterseer_2016_landraces_quality_count_table.sum()

In [ ]:
unterseer_2016_elite_quality_count_table=unterseer_2016_elite_quality_count.to_frame()
#unterseer_2016_elite_quality_count_table

In [ ]:
unterseer_2016_elite_quality_count_table.sum()

In [ ]:
mayer_2017_landraces_quality_count_table=mayer_2017_landraces_quality_count.to_frame()
#mayer_2017_landraces_quality_count_table

In [ ]:
mayer_2017_landraces_quality_count_table.sum()

In [ ]:
mayer_2020_dhlines_quality_count_table=mayer_2020_dhlines_quality_count.to_frame()
#mayer_2020_dhlines_quality_count_table

In [ ]:
mayer_2020_dhlines_quality_count_table.sum()

In [ ]:
# lists put together by hand from the count_tables above
PolyHighResolution_count=[527511, 560483, 527743, 523955]
MonoHighResolution_count=[521,387, 521,1058]
NoMinorHom_count=[7908,273,7912,15740]
CallRateBelowThreshold_count=[846,30,848,748]
OTV_count=[11392, 45387, 11395, 20258]
Other_count=[60954, 2572, 61023, 47683]

*Plot*

In [ ]:
labels = ['Unterseer_2016_landraces', 'Unterseer_2016_elite', 'Mayer_2017', 'Mayer_2020']

x = np.arange(len(labels))  # the label locations
width = 0.15  # the width of the bars

# Set position of bar on X axis
r2 = np.arange(4)
r3 = [x + width for x in r2]
r4 = [x + width for x in r3]
r5 = [x + width for x in r4]
r6 = [x + width for x in r5]

fig, ax = plt.subplots()

rects2 = ax.bar(r2, MonoHighResolution_count, width, label='MonoHighResolution')
rects3 = ax.bar(r3, NoMinorHom_count, width, label='NoMinorHom')
rects4 = ax.bar(r4, CallRateBelowThreshold_count, width, label='CallRateBelowThreshold')
rects5 = ax.bar(r5, OTV_count, width, label='OTV')
rects6 = ax.bar(r6, Other_count, width, label='Other')

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
        
autolabel(rects2)
autolabel(rects3)
autolabel(rects4)
autolabel(rects5)
autolabel(rects6)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('number of markers',fontsize='large', fontweight= "bold")
ax.set_title('Axiom Array 600k Marker Quality Category', fontsize='xx-large', fontweight= "bold")
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize='large', ha="left", fontweight= "bold")
ax.legend()


plt.show()
#plt.savefig('/Users/kschul38/Documents/tsinfer-project/figures/Stetter_phd_europe_maize_figures/count_all_minor_categories.png')

#### **1.9.5.3 Make a barplot for the comparison for the best quality categories vs. all other categories**

In [ ]:
labels = ['Unterseer_2016_landraces', 'Unterseer_2016_elite', 'Mayer_2017', 'Mayer_2020']

PolyHighResolution_count=[527511, 560483, 527743, 523955]
count_other=[(521+7908+846+11392+60954),(387+273+30+45387+2572),(521+7912+848+11395+61023),(1058+15740+748+20258+47683)]

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

# Set position of bar on X axis
f1 = np.arange(4)
f2 = [x + width for x in f1]


fig, ax = plt.subplots()
rects1 = ax.bar(f1, PolyHighResolution_count, width, label='PolyHighResolution')
rects2 = ax.bar(f2, count_other, width, label='Markers of other qualities')


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')
autolabel(rects1)
autolabel(rects2)


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('number of markers',fontsize='large', fontweight= "bold")
ax.set_title('Axiom Array 600k Marker Quality Category', fontsize='xx-large', fontweight= "bold")
ax.set_xticks(x)
ax.set_xticklabels(labels, fontsize='large', ha="left", fontweight= "bold")
ax.legend()


plt.show()
#plt.savefig('/Users/kschul38/Documents/tsinfer-project/figures/Stetter_phd_europe_maize_figures/marker_best_quality_categories_vs_all_other_categories.png')

#### **1.9.5.4 Make a barplot for all combinations that dont include Polyhigh at all**

*Count all quality marker combinations for each marker across all datasets*

In [ ]:
count_marker_qual_combis=merge_marker_all.apply(Counter, axis='columns').value_counts()

In [ ]:
#convert to dataframe
count_marker_qual_combis=count_marker_qual_combis.to_frame()
count_marker_qual_combis

In [ ]:
#convert number of combinations to list 
occurences=count_marker_qual_combis.values.tolist()

In [ ]:
#convert the index (description of combinations) into list of strings
index_count_marker_qual_combis=count_marker_qual_combis.index.values.tolist() 
index_count_marker_qual_combis=[str(i) for i in index_count_marker_qual_combis]

In [ ]:
index_count_marker_qual_combis

In [ ]:
#clean up the index, removing the Counter etc.
new_row_index=[]

In [ ]:
for row_index in index_count_marker_qual_combis:
    #print(row_index)
    new_index=row_index.replace("Counter(", "").replace("{", "").replace("})", "")
    new_row_index.append(new_index)
    #print(row_index)

In [ ]:
#new_row_index

*Make dataframe of the quality marker combination counts*

In [ ]:
#make dataframe & clean up dataframe 
call_qual_marker_combies = pd.DataFrame(occurences) #dataframe counts
call_qual_marker_combies['Combinations'] = new_row_index #add index 
call_qual_marker_combies=call_qual_marker_combies.rename(columns={0: "Counts"}) #remain column
call_qual_marker_combies
call_qual_marker_combies = call_qual_marker_combies[['Combinations','Counts']] #reorder the columns
call_qual_marker_combies

*Subset all rows that contain a substring in their index/description*

In [ ]:
#subset all rows that contain a substring in their index/description
contain_values = call_qual_marker_combies[call_qual_marker_combies['Combinations'].str.contains('PolyHighResolution')]
#contain_values

In [ ]:
#subset all rows that dont contain a substring in their index/description
contain_vaules_without_polyhigh=call_qual_marker_combies[~call_qual_marker_combies['Combinations'].str.contains('PolyHighResolution')]

In [ ]:
#sum over the subset dataframe 
print(contain_values.sum())

In [ ]:
#sum over the subset dataframe 
contain_vaules_without_polyhigh.sum()

In [ ]:
#sum to get number of total SNPs
593760+15682